# Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia

In this project, we will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](Falcon9_rocket_family.svg)

Falcon 9 first stage successful landing</br>
![](landing_1.gif)</br>

Examples of an unsuccessful landing</br>
![](crash.gif)</br>

Launch records</br>
![](falcon9-launches-wiki.png)</br>


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame

In [ ]:
# install required packages
!pip3 install beautifulsoup4
!pip3 install requests

In [150]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

In [170]:
# helper functions

def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]


def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out


def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on `9th June 2021`

In [152]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

### Step 1: Request the Falcon9 Launch Wiki page from its URL

First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.

In [171]:
# use requests.get() method with the provided static_url
# assign the response to a object
html_data = requests.get(static_url).text
# html_data

In [172]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(html_data, 'html5lib')

In [173]:
# Print the page title to verify if the BeautifulSoup object was created properly
soup.title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### Step 2: Extract all column/variable names from the HTML table header

Next, we want to collect all relevant column names from the HTML table header.

In [174]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')

In [176]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
#print(first_launch_table)
#print(first_launch_table.prettify())

You should able to see the columns names embedded in the table header elements `<th>` as follows:

```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```

Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [178]:
# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names

column_names = []

for name in first_launch_table.find_all('th'):
    print(name.text.strip(), extract_column_from_header(name))
    if (name is not None and len(name.text.strip()) > 0):
        column_names.append(extract_column_from_header(name))

Flight No. Flight No.
Date andtime () Date and time ( )
 
Launch site Launch site
Payload Payload
Payload mass Payload mass
Orbit Orbit
Customer Customer
Launchoutcome Launch outcome
 
1 None
2 None
3 None
4 None
5 None
6 None
7 None


In [179]:
column_names

['Flight No.',
 'Date and time ( )',
 'Launch site',
 'Payload',
 'Payload mass',
 'Orbit',
 'Customer',
 'Launch outcome',
 None,
 None,
 None,
 None,
 None,
 None,
 None]

## Step 3: Create a data frame by parsing the launch HTML tables

We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe

In [180]:
launch_dict = dict.fromkeys(column_names)
launch_dict

{'Flight No.': None,
 'Date and time ( )': None,
 'Launch site': None,
 'Payload': None,
 'Payload mass': None,
 'Orbit': None,
 'Customer': None,
 'Launch outcome': None,
 None: None}

In [181]:
# Remove an irrelvant column
del launch_dict['Date and time ( )']
launch_dict

{'Flight No.': None,
 'Launch site': None,
 'Payload': None,
 'Payload mass': None,
 'Orbit': None,
 'Customer': None,
 'Launch outcome': None,
 None: None}

In [182]:
# Add empty list as value to each key
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

launch_dict

{'Flight No.': [],
 'Launch site': [],
 'Payload': [],
 'Payload mass': [],
 'Orbit': [],
 'Customer': [],
 'Launch outcome': [],
 None: None,
 'Version Booster': [],
 'Booster landing': [],
 'Date': [],
 'Time': []}

In [183]:
# get rid of None column
launch_dict.pop(None)
launch_dict

{'Flight No.': [],
 'Launch site': [],
 'Payload': [],
 'Payload mass': [],
 'Orbit': [],
 'Customer': [],
 'Launch outcome': [],
 'Version Booster': [],
 'Booster landing': [],
 'Date': [],
 'Time': []}

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.

Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.

In [194]:
# SKIP THIS SECTION. This is to view and test the row and cell values..

# There are multiple tables in the page.
# extract each tables, 
# validate if the first col is flight number before  

extracted_row = 0

for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
    #print(table_number, table)
    for rows in table.find_all('tr'):
        #print(rows)
        # check to see if first table heading is as number corresponding
        # to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                flag = flight_number.isdigit()                
        else:
            flag = False
            
        # if flag = True, or skip all
        if flag:
            extracted_row += 1
            # get table element
            row = rows.find_all('td')        
            '''
            row[0] = Time, Date, Month, Year
            row[1] = Booster version
            row[2] = Launch Site
            row[3] = Payload
            row[4] = Payload mass
            row[5] = Orbit
            row[6] = Customer
            row[7] = Launch outcome
            row[8] = Booster landing
            '''
            '''
            print('flight number:', flight_number)
            print('Flight date:', row[0].text.strip())   
            print('Booster version:', row[1].a.string)
            print('Launch site:', row[2].a.string)
            print('Payload:', row[3].a.string)
            print('Payload mass:', row[4].text.strip())
            print('Orbit:', row[5].a.string)
            print('Customer:', row[6].a.string)
            print('Launch outcome:', row[7].text.strip())
            print('Booster landing:', row[8].text.strip())
 
            '''
            '''
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            print('flight_number:', flight_number)
            launch_dict['Flight No.'].append(flight_number)
                        
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            #datatimelist=date_time(row[0])
            #date = datatimelist[0].strip(',')
            date = row[0].text.strip()
            print('Flight date:', date)   
            launch_dict['Date'].append(date)

            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            #time = datatimelist[1]
            #print('time:', time)            
            #launch_dict['Time'].append(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            #bv=booster_version(row[1])
            #if not(bv):
            #    bv=row[1].a.string
            #print('booster_version:', bv)
            
            bv = row[1].a.string
            print('Booster version:', bv)
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            print('launch_site:', launch_site)
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            print('payload:', payload)
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            #payload_mass = get_mass(row[4])
            payload_mass = row[4].text.strip()
            print('payload_mass:', payload_mass)
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            print('orbit:', orbit)
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`               
            customer = row[6].a.string if row[6].a else None
            print('customer:', customer)
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            print('launch_outcome:', launch_outcome.strip())
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            # booster_landing = landing_status(row[8])
            
            booster_landing = row[8].text.strip()
            print('booster_landing:', booster_landing)
            launch_dict['Booster landing'].append(booster_landing)
            
            print("*")
            '''
print('extracted_row:', extracted_row)    

extracted_row: 121


In [195]:
extracted_row = 0

print(launch_dict)

#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
    # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            print('flight_number:', flight_number)
            launch_dict['Flight No.'].append(flight_number)
                        
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            print('date:', date)            
            launch_dict['Date'].append(date)

            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            print('time:', time)            
            launch_dict['Time'].append(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print('booster_version:', bv)
            launch_dict['Version Booster'].append(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            print('launch_site:', launch_site)
            launch_dict['Launch site'].append(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            print('payload:', payload)
            launch_dict['Payload'].append(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            print('payload_mass:', payload_mass)
            launch_dict['Payload mass'].append(payload_mass)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            print('orbit:', orbit)
            launch_dict['Orbit'].append(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`               
            customer = row[6].a.string if row[6].a else None
            print('customer:', customer)
            launch_dict['Customer'].append(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            print('launch_outcome:', launch_outcome)
            launch_dict['Launch outcome'].append(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            print('booster_landing:', booster_landing)
            launch_dict['Booster landing'].append(booster_landing)
            
            print("---------------------------------------")
            
print('extracted_row:', extracted_row)    

{'Flight No.': ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62', '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73', '74', '75', '76', '77', '78', '79', '80', '81', '82', '83', '84', '85', '86', '87', '88', '89', '90', '91', '92', '93', '94', '95', '96', '97', '98', '99', '100', '101', '102', '103', '104', '105', '106', '107', '108', '109', '110', '111', '112', '113', '114', '115', '116', '117', '118', '119', '120', '121', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '4

After you have fill in the parsed launch record values into launch_dict, you can create a dataframe from it.

In [185]:
#launch_dict

In [187]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

In [188]:
df.shape

(121, 11)

In [189]:
df.head()

,Flight No.,Launch site,Payload,Payload mass,Orbit,Customer,Launch outcome,Version Booster,Booster landing,Date,Time
0,1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,F9 v1.0B0003.1,Failure,4 June 2010,18:45
1,2,CCAFS,Dragon,0,LEO,NASA,Success,F9 v1.0B0004.1,Failure,8 December 2010,15:43
2,3,CCAFS,Dragon,525 kg,LEO,NASA,Success,F9 v1.0B0005.1,No attempt\n,22 May 2012,07:44
3,4,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,F9 v1.0B0006.1,No attempt,8 October 2012,00:35
4,5,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,F9 v1.0B0007.1,No attempt\n,1 March 2013,15:10


Export the data frame to csv file

In [190]:
df.to_csv('spacex_web_scraped.csv', index=False)